In [1]:
import random
import json
import pickle

import nltk
nltk.download('all')
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

import numpy as np

lemmatizer = WordNetLemmatizer()

intents = json.loads(open("/content/intents.json").read())

words = []
classes = []
documents = []

ignore_letters = ["?", "!", ".", ","]

for intent in intents["intents"]:
	for pattern in intent["patterns"]:
		word_list = nltk.word_tokenize(pattern)
		words.extend(word_list)
		documents.append((word_list, intent["tag"]))

		if intent["tag"] not in classes:
			classes.append(intent["tag"])
words = [lemmatizer.lemmatize(word)
		for word in words if word not in ignore_letters]

words = sorted(set(words))
classes = sorted(set(classes))

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))
dataset = []
template = [0]*len(classes)

for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]

    # Create a bag of words vector
    for word in words:
        bag.append(1 if word in word_patterns else 0)

    # Ensure output_row matches the number of classes
    output_row = list(template)
    output_row[classes.index(document[1])] = 1

    # Append the consistent feature-label pair
    dataset.append([bag, output_row])

# Shuffle the dataset and check shapes
random.shuffle(dataset)

# Ensure all bags and labels have consistent lengths
dataset = [(np.array(item[0], dtype=np.float32), np.array(item[1], dtype=np.float32)) for item in dataset]
dataset = np.array(dataset, dtype=object)

# Extract training data
train_x = np.array([item[0] for item in dataset])
train_y = np.array([item[1] for item in dataset])

model = Sequential()
model.add(Dense(256, input_shape=(len(train_x[0]),),activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))


sgd = SGD(learning_rate=0.01,momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',optimizer=sgd, metrics=['accuracy'])

hist = model.fit(np.array(train_x), np.array(train_y),epochs=200, batch_size=5, verbose=1)

model.save("chatbot_model.h5", hist)
print("Done!")


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_r

Epoch 1/200
111/111 [==============================] - 2s 10ms/step - loss: 4.5827 - accuracy: 0.0054
Epoch 2/200
111/111 [==============================] - 3s 28ms/step - loss: 4.5373 - accuracy: 0.0236
Epoch 3/200
111/111 [==============================] - 3s 31ms/step - loss: 4.5058 - accuracy: 0.0272
Epoch 4/200
111/111 [==============================] - 2s 16ms/step - loss: 4.4720 - accuracy: 0.0345
Epoch 5/200
111/111 [==============================] - 1s 9ms/step - loss: 4.4172 - accuracy: 0.0363
Epoch 6/200
111/111 [==============================] - 1s 5ms/step - loss: 4.3817 - accuracy: 0.0417
Epoch 7/200
111/111 [==============================] - 1s 6ms/step - loss: 4.3224 - accuracy: 0.0508
Epoch 8/200
111/111 [==============================] - 1s 6ms/step - loss: 4.2485 - accuracy: 0.0653
Epoch 9/200
111/111 [==============================] - 1s 5ms/step - loss: 4.1382 - accuracy: 0.1071
Epoch 10/200
111/111 [==============================] - 1s 6ms/step - loss: 4.0040 - ac

Python 3.10.12
